In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from twilio.rest import Client
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Email (Gmail) config
EMAIL_SENDER = "samplemail15856@gmail.com"
EMAIL_APP_PASSWORD = "xnmdwagpqonytfex"  # 16-char app password
EMAIL_SMTP_SERVER = "smtp.gmail.com"
EMAIL_SMTP_PORT = 587
ADMIN_EMAILS = ["vanshisethi1815@gmail.com"]  # Replace with your admin emails

# Twilio config - replace with your own data
TWILIO_ACCOUNT_SID = "ACc4ceee01d08ffc10530bd8de76144d29"
TWILIO_AUTH_TOKEN = "0443151df8f01c49bbd9e37911fd6a15"
TWILIO_PHONE_NUMBER = "+15856394422"  # Your Twilio phone number with country code
ADMIN_PHONE_NUMBERS = ["+917988021650","+919416451692"]  # Replace with recipient phone numbers with country code

class AlertSystem:
    def __init__(self):
        self.twilio_client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

    def send_email_alert(self, location, probability, risk_level):
        subject = f"🚨 Rockfall Alert: {risk_level} risk detected at {location}"
        body = (f"Alert: Rockfall risk detected at {location}.\n"
                f"Risk Level: {risk_level}\n"
                f"Probability: {probability:.2%}\n"
                "Immediate safety measures should be taken.")
        message = MIMEMultipart()
        message['From'] = EMAIL_SENDER
        message['To'] = ", ".join(ADMIN_EMAILS)
        message['Subject'] = subject
        message.attach(MIMEText(body, 'plain'))

        try:
            with smtplib.SMTP(EMAIL_SMTP_SERVER, EMAIL_SMTP_PORT) as server:
                server.starttls()
                server.login(EMAIL_SENDER, EMAIL_APP_PASSWORD)
                server.sendmail(EMAIL_SENDER, ADMIN_EMAILS, message.as_string())
            logger.info(f"Email alert sent to: {ADMIN_EMAILS}")
        except Exception as e:
            logger.error(f"Failed to send email alert: {e}")

    def send_sms_alert(self, location, probability, risk_level):
        text = (f"🚨 Rockfall Alert!\nLocation: {location}\n"
                f"Risk Level: {risk_level}\nProbability: {probability:.2%}\n"
                "Please take immediate precautions!")
        for phone in ADMIN_PHONE_NUMBERS:
            try:
                message = self.twilio_client.messages.create(
                    body=text,
                    from_=TWILIO_PHONE_NUMBER,
                    to=phone
                )
                logger.info(f"SMS sent to {phone}. SID: {message.sid}")
            except Exception as e:
                logger.error(f"Failed to send SMS to {phone}: {e}")

    def trigger_alert(self, location, probability, threshold=0.7):
        if probability >= threshold:
            risk_level = "HIGH"
        elif probability >= 0.5:
            risk_level = "MEDIUM"
        else:
            logger.info(f"No alert triggered. Probability: {probability:.2%}")
            return
        logger.warning(f"Triggering {risk_level} alert for {location}")
        self.send_email_alert(location, probability, risk_level)
        self.send_sms_alert(location, probability, risk_level)

if __name__ == "__main__":
    alert = AlertSystem()
    alert.trigger_alert("Jharkhand Mine", 0.75)


INFO:__main__:Email alert sent to: ['vanshisethi1815@gmail.com']
INFO:twilio.http_client:-- BEGIN Twilio API Request --
INFO:twilio.http_client:POST Request: https://api.twilio.com/2010-04-01/Accounts/ACc4ceee01d08ffc10530bd8de76144d29/Messages.json
INFO:twilio.http_client:Headers:
INFO:twilio.http_client:Content-Type : application/x-www-form-urlencoded
INFO:twilio.http_client:Accept : application/json
INFO:twilio.http_client:User-Agent : twilio-python/9.8.0 (Windows AMD64) Python/3.12.7
INFO:twilio.http_client:X-Twilio-Client : python-9.8.0
INFO:twilio.http_client:Accept-Charset : utf-8
INFO:twilio.http_client:-- END Twilio API Request --
INFO:twilio.http_client:Response Status Code: 201
INFO:twilio.http_client:Response Headers: {'Content-Type': 'application/json;charset=utf-8', 'Content-Length': '932', 'Connection': 'keep-alive', 'Date': 'Sat, 13 Sep 2025 09:28:52 GMT', 'Twilio-Concurrent-Requests': '1', 'Twilio-Request-Id': 'RQb0fe711a79053ead0312e6f2b443bbdd', 'Twilio-Request-Durat